## Clasificación Gausiana con Bayes

Para trabajar con Bayes en machine learning, vamos a definir la nomenclatura de la del teorema de Bayes para que sea más comprensible bajo nuestros estandares de ML. 

Pordemos re-imaginar la fórmula de la siguiente forma:

$$P(clase|datos)=\frac{P(datos|clase)*P(clase)}{P(datos)}$$

Nuestra definición de Bayes sirve igualmente para clasificación binaria, como para clasificación multi-clase. Vamos a utilizar nuestro dataset de Iris para ejemplificar la clasificación multi-clase. 

Vamos a utilizar los datos directamente de Sklearn. Para simplificar el ejercicio, vamos a utilizar solo 2 features: el largo y el ancho de sépalo.

In [ ]:
from sklearn import datasets
import numpy as np
import pandas as pd
from scipy.stats import norm
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = iris.data[:,:2]
y = iris.target

list(iris.target_names)

## Probabilidad previa: P(Clase) o P(y = c)

Es la probabilidad de una clase es la frecuencia de las instancias de una clase dividido sobre el total de instancias en el dataset.

- P(setosa) = # clases setosa / Total de Datos
- P(versicolor) = # clases versicolor / Total de Datos
- P(virginica) # clases virginica / Total de Datos

En este caso todas las clases obtienen una probabilidad de 0.33, porque hay 50 ejemplos de cada tipo. 

In [ ]:
p_setosa = np.sum((y == 0) * 1) / len(y)
p_versicolor = np.sum((y == 1) * 1) / len(y)
p_virginica = np.sum((y == 2) * 1) / len(y)

print("P(setosa)",p_setosa)
print("P(versicolor)",p_versicolor)
print("P(virginica)",p_virginica)

## Probabilidad: P(datos|clase) o p(x|y = c) 

Vamos a hacer una pequena modificacion a nuestra formula, para estimar P(datos|clase) en funcion de una distribucion Gaussiana. Prácticamente asumimos que los datos X pertenes a una distribucion normal que esta descrita por una media y una desviacion estandard obtenida de los datos, para cada clase. La prediccion consta en determinar si los datos x pertenece a una distribucion u a otra. 

La funcion de densidad gaussiana, también es llamada como Probability Density Function (PDF)

La fórmula de densidad Gaussiana estaria definida de la siguiente forma:

<img src="img/nd.png" />

Ahora, vamos a eliminar el termino normalizador, y vamos a calcular la probabilidad bajo la siguiente definición:

<img src="img/bayes_g.png" />

Donde ϕ es la función de la densidad Gaussiana. Vamos a ejecutar ϕ(x1|mu,sig)*p(y) por cada clase y (osea tres veces). Vamos a seleccionar la probabilidad más alta de los 3 cálculos. Esa probabilidad, sera de la clase más probable.

El metodo predecir que tenemos abajo, usa la funcion *norm.PDF* de la librería scipy.stats que implementa la función gaussiana.

In [ ]:
# Vamos a crear un dataframe para genenrar un mejor filtrado de las medias y varianzas de cada clase
df = pd.DataFrame({"largo_sepalo": X[:,0], "ancho_sepalo": X[:,1], "clase": y})

# caculamos la media, desviacion estandard de cada feature xi segun la clase
medias = np.split(df.groupby('clase').mean().values,[1,2])
desviaciones = np.split(df.groupby('clase').std().values,[1,2], axis = 0)

# generamos el arreglo de las probabilidades previas
prob_clases = np.array([p_setosa, p_versicolor, p_virginica])

In [ ]:
def predecir(x_,medias_,desviaciones_,probs_): 
    scores = []
    num_clases = len(probs_)
    
    # ϕ(x1|mu,sig)*p(y) * ϕ(x1|mu,sig)*p(y) * ϕ(x1|mu,sig)*p(y)
    for p in range(num_clases):
        valor = (norm.pdf(x = x_[0], loc = medias_[p][0][0], scale = desviaciones_[p][0][0] )  
                * norm.pdf(x = x_[1], loc = medias_[p][0][1], scale = desviaciones_[p][0][1] ) 
                * probs_[p])
        
        scores.append(valor)
             
    return np.argmax(scores)

In [ ]:
y_prima = predecir([5.1,3.5], medias, desviaciones, prob_clases)
print("una planta con los valores 5.1,3.5, pertencen a la clase", iris.target_names[y_prima])

In [ ]:
# predecir todo el dataset

y_prima = [predecir(x, medias, desviaciones, prob_clases) for x in X]

In [ ]:
# vamos a ver el accuracy de la prediccion.
acc = accuracy_score(y, y_prima)

print("La exactitud de la predicción es de:", acc)